# Proyecto Final - Aprendizaje Automático - Diego Estrada

## Problema de interés: Calidad de agua en el Río de la Plata

Vamos a trabajar y cargar data sets desde el periodo 2013 al 2024.

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

#Ruta de los dataset
ruta_base = 'C:\\Users\\destrada\\Ciencia_datos_2A1C\\Cookiecutter_Proyecto_Final\\data\\interim\\'

#Verificamos si la ruta existe
if os.path.exists(ruta_base):
    #Cargamos y normalizamos los datasets
    archivos = [
        'agc_y_riodelaplata_2013.csv',
        'agc_y_riodelaplata_2014.csv',
        'agc_y_riodelaplata_2015.csv',
        'agc_y_riodelaplata_2016.csv',
        'agc_y_riodelaplata_2017.csv',
        'agc_z_riodelaplata_2018.csv',
        'agc_z_riodelaplata_2019.csv',
        'agc_z_riodelaplata_2020.csv',
        'agc_z_riodelaplata_2021.csv',
        'agc_y_riodelaplata_2022.csv',
        'agc_y_riodelaplatal_2023.csv',
        'agc_z_riodelaplata_2024.csv'
    ]
    
    dataframes = []
    for archivo in archivos:
        df = pd.read_csv(ruta_base + archivo, encoding='latin1', sep=';')
        #Convertir los nombres de columnas a minúsculas para estandarizarlos y poder unificar correctamente.
        df.columns = map(str.lower, df.columns)
        dataframes.append(df)
    
    #Unificamos los datasets
    df_final = pd.concat(dataframes, ignore_index=True)
else:
    print("La ruta especificada no existe.")

#Mostramos las primeras filas
df_final.head()

,orden,sitios,gobierno_local,codigo,fecha,año,campaña,tem_agua,od,ph,...,cr_total_mg_l,cd_total_mg_l,clorofila_a_ug_l,microcistina_ug_l,solidos_totales_mg_l,municipio,p_de_ortof_mg_l,hidr_deriv_del_petroleo_ug_l,escherichia_coli_ufc_100ml,hidrocarburos_deriv_del_petróleo_ug_l
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Canal Villanueva y RÃ­o LujÃ¡n,Tigre,TI001,28/8/2013,2013.0,invierno,10.3,0.7,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,RÃ­o Lujan y Arroyo CaraguatÃ¡,Tigre,TI006,28/8/2013,2013.0,invierno,10.5,0.5,7.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Queremos conocer la cantidad de registros en el DataFrame Unificado
num_registros = df_final.shape[0]
print("Cantidad de registros: " + str(num_registros))

#Cantidad de columnas
num_columnas = df_final.shape[1]
print("Cantidad de columnas: " + str(num_columnas))

Cantidad de registros: 2428
Cantidad de columnas: 39


##  Preprocesamiento de Datos (Data Preprocessing)
Vamos a trabajar en el proceso de ETL, después que unificamos los datos, eliminaremos algunas columnas que no necesitamos analizar, lo que también mejorará el rendimiento del modelo que vamos a desarrollar.

In [6]:
#Validamos si tenemos valores nulos en cada columnas
valores_nulos = df_final.isnull().sum()
print(valores_nulos)

orden                                     763
sitios                                    763
gobierno_local                           1918
codigo                                    763
fecha                                     763
año                                       779
campaña                                   779
tem_agua                                  790
od                                       1072
ph                                        910
colif_totales_ufc_100ml                  1269
colif_fecales_ufc_100ml                  1682
escher_coli_ufc_100ml                    1023
nitrato_mg_l                              937
nh4_mg_l                                  943
dqo_mg_l                                 1025
ica                                      1111
calidad_de_agua                          1150
tem_aire                                 1585
olores                                   1420
color                                    1439
espumas                           

In [7]:
#Vamos a eliminar filas duplicadas, para quitar todas las filas que sean completamente vacias y repetidas.
df_final = df_final.drop_duplicates()

In [8]:
#Vamos a quitar columnas que no serán relevante para el modelo de aprendizaje automatico
columnas_a_eliminar = ['orden', 'sitios', 'gobierno_local', 'codigo', 'fecha', 'municipio','escherichia_coli_ufc_100ml',
                       'hidrocarburos_deriv_del_petróleo_ug_l','hidr_deriv_del_petroleo_ug_l','p_de_ortof_mg_l','solidos_totales_mg_l']

#Eliminamos filas con valores nulos en las columnas 'años' y 'campaña'
#porque hay dataset que tiene filas completamente vacias y en el analisis de valores nulos los cuenta.
df_final = df_final.dropna(subset=['año', 'campaña'])

#Aplicamos la condición para validar
if all(col in df_final.columns for col in columnas_a_eliminar):
    df_final = df_final.drop(columns=columnas_a_eliminar)
else:
    print("Una o más columnas no existen.")

Luego de eliminar las columnas que no aportaban información relevante para el análisis, se identificaron otras que deberían contener exclusivamente valores numéricos de tipo flotante. Sin embargo, además de presentar valores nulos, algunas de estas columnas incluyen descripciones como "No se midió", "en obra" o "no muestreó", lo que compromete la consistencia de los datos. Para garantizar la integridad del análisis, estos valores serán reemplazados por la mediana correspondiente de cada columna.

In [10]:
#Con este ciclo, iteramos para que cada columna, reemplazamos valores y calcule la mediana
for col in ['od', 'ph', 'tem_agua','colif_totales_ufc_100ml','escher_coli_ufc_100ml','nitrato_mg_l','dqo_mg_l','dbo_mg_l','cr_total_mg_l',
            'colif_fecales_ufc_100ml','fosf_ortofos_mg_l','hidr_deriv_petr_ug_l','tem_aire','ica','enteroc_ufc_100ml','nh4_mg_l','p_total_l_mg_l',
           'turbiedad_ntu','cd_total_mg_l','clorofila_a_ug_l','microcistina_ug_l']:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')
    df_final[col] = df_final[col].fillna(df_final[col].median())

Luego de tratar las columnas numéricas con valores nulos, procederemos ahora a corregir los valores nulos para las columnas categóricas.

In [12]:
#Columnas que queremos procesar
columnas = ['olores', 'color', 'espumas', 'mat_susp']

#Reemplazamos "no se midió" y valores nulos por la moda de cada columna
for columna in columnas:
    moda = df_final[columna].mode().iloc[0]  # Obtiene la moda (valor más frecuente)
    df_final[columna] = df_final[columna].replace("no se midió", pd.NA)
    df_final[columna] = df_final[columna].fillna(moda)

In [13]:
#Volvemos a validar si tenemos valores nulos en cada columna
valores_nulos = df_final.isnull().sum()
print("Valores nulos por columna:\n", valores_nulos)

#Mostramos la cantidad total de registros
num_registros = df_final.shape[0]
print("\nCantidad de registros:", num_registros)

#Mostramos el tipo de dato de cada columna
tipos_de_dato = df_final.dtypes
print("\nTipo de dato por columna:\n", tipos_de_dato)

Valores nulos por columna:
 año                          0
campaña                      0
tem_agua                     0
od                           0
ph                           0
colif_totales_ufc_100ml      0
colif_fecales_ufc_100ml      0
escher_coli_ufc_100ml        0
nitrato_mg_l                 0
nh4_mg_l                     0
dqo_mg_l                     0
ica                          0
calidad_de_agua            372
tem_aire                     0
olores                       0
color                        0
espumas                      0
mat_susp                     0
enteroc_ufc_100ml            0
p_total_l_mg_l               0
fosf_ortofos_mg_l            0
dbo_mg_l                     0
turbiedad_ntu                0
hidr_deriv_petr_ug_l         0
cr_total_mg_l                0
cd_total_mg_l                0
clorofila_a_ug_l             0
microcistina_ug_l            0
dtype: int64

Cantidad de registros: 1649

Tipo de dato por columna:
 año                         object

El siguiente paso consiste en convertir las variables categóricas a formato numérico. Esta transformación seria necesaria porque los algoritmos de aprendizaje automático requieren datos numéricos para funcionar correctamente y alcanzar un mejor rendimiento.
En particular, las columnas olores, color, espumas y mat_susp contienen valores categóricos como "Ausente", "ausente", "Presente" y "presente". Estos valores serán estandarizados y luego codificados numéricamente.
Además, se identificaron filas en las que todas estas columnas contienen el valor "no se midió". Dado que en estos casos no se dispone de información útil en ninguna de las variables mencionadas, se procederá a eliminar dichas filas del conjunto de datos para mantener la calidad del análisis.

In [27]:

#Columnas categóricas a procesar
columnas_categoricas = ['olores', 'color', 'espumas', 'mat_susp']

#Eliminamos filas donde todas esas columnas tienen el valor 'no se midió'
df_final = df_final[~(
    (df_final['olores'].str.lower() == 'no se midió') &
    (df_final['color'].str.lower() == 'no se midió') &
    (df_final['espumas'].str.lower() == 'no se midió') &
    (df_final['mat_susp'].str.lower() == 'no se midió')
)]

#Estandarizamos los valores a minúsculas
df_final[columnas


SyntaxError: invalid non-printable character U+00A0 (596811306.py, line 6)

In [14]:
#Queremos conocer la cantidad de registros en el DataFrame Unificado y como quedo con todo el tratamiento que realizamos
num_registros = df_final.shape[0]
print("Cantidad de registros: " + str(num_registros))

#Cantidad de columnas
num_columnas = df_final.shape[1]
print("Cantidad de columnas: " + str(num_columnas))

Cantidad de registros: 1649
Cantidad de columnas: 28
